# 个性化推荐
本项目使用文本卷积神经网络，并使用[`MovieLens`](https://grouplens.org/datasets/movielens/)数据集完成电影推荐的任务。



推荐系统在日常的网络应用中无处不在，比如网上购物、网上买书、新闻app、社交网络、音乐网站、电影网站等等等等，有人的地方就有推荐。根据个人的喜好，相同喜好人群的习惯等信息进行个性化的内容推荐。比如打开新闻类的app，因为有了个性化的内容，每个人看到的新闻首页都是不一样的。

这当然是很有用的，在信息爆炸的今天，获取信息的途径和方式多种多样，人们花费时间最多的不再是去哪获取信息，而是要在众多的信息中寻找自己感兴趣的，这就是信息超载问题。为了解决这个问题，推荐系统应运而生。

协同过滤是推荐系统应用较广泛的技术，该方法搜集用户的历史记录、个人喜好等信息，计算与其他用户的相似度，利用相似用户的评价来预测目标用户对特定项目的喜好程度。优点是会给用户推荐未浏览过的项目，缺点呢，对于新用户来说，没有任何与商品的交互记录和个人喜好等信息，存在冷启动问题，导致模型无法找到相似的用户或商品。

为了解决冷启动的问题，通常的做法是对于刚注册的用户，要求用户先选择自己感兴趣的话题、群组、商品、性格、喜欢的音乐类型等信息，比如豆瓣FM：
<img src="assets/IMG_6242_300.PNG"/>

In [1]:

import collections
from collections import Counter
import random
import math

import pandas as pd
import numpy as np
import scipy.sparse as ss
from scipy.spatial.distance import jaccard, cosine, hamming

import tensorflow as tf

import os
import pickle
import re
from tensorflow.python.ops import math_ops

from sklearn.externals.joblib import dump, load
import utils

In [10]:
data_dir = '../data/'
output_dir = '../output/'
get_distance = utils.get_distance
get_user_sim = utils.get_user_sim
get_movie_sim = utils.get_event_sim

## 先来看看数据

本项目使用的是MovieLens 1M 数据集，包含6000个用户在近4000部电影上的1亿条评论。

数据集分为三个文件：用户数据users.dat，电影数据movies.dat和评分数据ratings.dat。

### 评分数据
分别有用户ID、电影ID、评分和时间戳等字段。

数据中的格式：UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

In [11]:
user_movie_rating = load(output_dir + 'user_movie_rating.joblib.gz')
user_movie_rating.shape

(6041, 3953)

评分字段Rating就是我们要学习的targets，时间戳字段我们不使用。

### 用户数据
分别有用户ID、性别、年龄、职业ID和邮编等字段。

数据中的格式：UserID::Gender::Age::Occupation::Zip-code

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"



In [12]:
users = load(output_dir + 'users.joblib.gz')
users.index = users.pop('UserID')
users.head()

,Gender_0,Gender_1,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,Age_6,OccupationID_0,...,Zip-code_90,Zip-code_91,Zip-code_92,Zip-code_93,Zip-code_94,Zip-code_95,Zip-code_96,Zip-code_97,Zip-code_98,Zip-code_99
UserID,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
print('getting the distance')
num_users = users.index.max()+1
# 生成数值型和类别型数据
users_num = users.loc[:,'Gender_0':]
users_cat = users.loc[:,'Gender_0':]
# 生成空的distance, 默认为1
user_distance = np.ones((num_users, num_users), dtype=np.float64)
# 生成user_index
u_index = users.index
# 生成1/10的界限, 用于输出进度
num_users010 = num_users//10
for i,u1 in enumerate(u_index):
    # 显示进度
    if i%100 == 0: print(i)
    # 对角线距离为0
    user_distance[u1,u1] = 0
    for u2 in u_index[i+1:]:
        # 计算距离, 并对称赋值
        dis = get_distance(u1,u2,users_num,users_cat)
        #sim_d = 1 - get_distance(u1,u2,users_num,users_cat)
        #sim_r = get_user_sim(u1,u2,user_movie_rating)
        print(sim_d, sim_r)
        user_distance[u1,u2] = dis
        user_distance[u2,u1] = dis

getting the distance
0
0.0 0.55123802894
0.0 0.374425990787
0.0 0.0557888839309
0.0 0.789779898357
0.207142857143 1.01618596703
0.0 0.0484705283653
0.0 0.732217764447
0.0 1.81274472636
0.207142857143 9.80889865625
0.207142857143 1.52356060013
0.0 0.116257774417
0.0 0.777857136482
0.0 0.0707376639633
0.0 0.377395955455
0.207142857143 0.0639785352846
0.0 1.21576965616
0.207142857143 6.66693344977
0.69 5.1674083556
0.0 0.00270442037465
0.0 0.311494868406
0.0 1.10286289521
0.0 1.15491562024
0.207142857143 0.988014354846
0.0 0.722781611079
0.0 4.2299986941
0.0 0.0291821956933
0.207142857143 1.20026958992
0.0 0.0753094654381
0.207142857143 0.688448205785
0.0 0.280485674422
0.207142857143 0.155791646128
0.0 6.58200857827
0.207142857143 1.98158266169
0.0 0.849454160127
0.0 2.17541388431
0.207142857143 0.223776071504
0.207142857143 0.825999899055
0.0 0.372458224439
0.0 0.18031571176
0.207142857143 0.0773840961822
0.0 0.64187796264
0.0 0.160846018909
0.0 3.31748065046
0.207142857143 2.2922894055

KeyboardInterrupt: 

### 电影数据
分别有电影ID、电影名和电影风格等字段。

数据中的格式：MovieID::Title::Genres

- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western


In [5]:
movies = load(output_dir + 'movies.joblib.gz')
movies.index = movies.pop('MovieID')
movies.head()

,Genres_Animation,Genres_Children's,Genres_Comedy,Genres_Adventure,Genres_Fantasy,Genres_Romance,Genres_Drama,Genres_Action,Genres_Crime,Genres_Thriller,...,Sent_vec_374,Sent_vec_375,Sent_vec_376,Sent_vec_377,Sent_vec_378,Sent_vec_379,Sent_vec_380,Sent_vec_381,Sent_vec_382,Sent_vec_383
MovieID,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.037464,-0.066699,-0.186861,0.096069,0.065446,0.030688,0.08348,-0.078668,0.05473,-0.102692


In [15]:
print('getting the distance')
num_movies = movies.index.max()+1
# 生成数值型和类别型数据
movies_num = movies.loc[:,'Genres_Animation':'Genres_Western']
movies_cat = movies.loc[:,'Title_yeay':]
# 生成空的distance, 默认为1
movie_distance = np.ones((num_movies, num_movies), dtype=np.float64)
# 生成movie_index
m_index = movies.index
for i,m1 in enumerate(m_index):
    # 显示进度
    if i%100 == 0: print(i)
    # 对角线距离为0
    movie_distance[m1,m1] = 0
    for m2 in m_index[i+1:]:
        # 计算距离, 并对称赋值
        dis = get_distance(m1,m2,movies_num,movies_cat)
        sim_d = 1 - get_distance(m1,m2,movies_num,movies_cat)
        sim_r = get_movie_sim(m1,m2,user_movie_rating)
        print(sim_d,':',sim_r)
        movie_distance[m1,m2] = dis
        movie_distance[m2,m1] = dis

getting the distance
0
0.203100775194 : -0.114727280814
0.247021513138 : -0.17846812203
0.247021513138 : -0.19196104336
0.347449122553 : -0.275086496274
0.0031007751938 : 0.158337106328
0.248049749472 : -0.0494851511646
0.247021513138 : -0.393241616234
0.0031007751938 : -0.622066046613
0.0031007751938 : 0.0242121620869
0.20207253886 : 0.199529369158
0.246195080197 : -0.597763030628
0.49299872375 : -0.145373997638
0.0031007751938 : 0.0542190609537
0.0031007751938 : -0.465453243078
0.0031007751938 : 0.15457872483
0.0020725388601 : 0.349112630781
0.0031007751938 : -0.0827408397948
0.347656267433 : -0.508895369365
0.0031007751938 : -0.514867584353
0.203100775194 : 0.076552473482
0.0031007751938 : 0.0123378828958
0.0031007751938 : -0.538601610851
0.0031007751938 : -0.208797169707
0.0020725388601 : -0.0176945526349
0.0031007751938 : 0.194074421576
0.0020725388601 : -0.274362612038
0.0031007751938 : 0.293801860025
0.001246105919 : 0.337757942792
0.00103896103896 : -0.238741764334
0.0031007751

KeyboardInterrupt: 

## 开始推荐电影
使用生产的用户特征矩阵和电影特征矩阵做电影推荐

In [17]:
class get_sim_dis(object):
    def __init__(self, k):
        self.k = k
        self.data_dir = '../data/'
        self.output_dir = '../output/'
        self.user_movie_rating = load(self.output_dir + 'user_movie_rating.joblib.gz')
        if k == 'u_s': self.get_u_s()
        elif k == 'u_d': self.get_u_d()
        elif k == 'm_s': self.get_m_s()
        elif k == 'm_d': self.get_m_d()
            
    def get_u_s(self,):
        users = load(self.output_dir + 'users.joblib.gz')
        users.index = users.pop('UserID')
        print('getting the distance')
        num_users = users.index.max()+1
        # 生成数值型和类别型数据
        users_num = users.loc[:,'Gender_0':]
        users_cat = users.loc[:,'Gender_0':]
        # 生成空的distance, 默认为1
        user_distance = np.ones((num_users, num_users), dtype=np.float64)
        # 生成user_index
        u_index = users.index
        for i,u1 in enumerate(u_index):
            # 显示进度
            if i%100 == 0: print(i)
            # 对角线距离为0
            user_distance[u1,u1] = 0
            for u2 in u_index[i+1:]:
                # 计算距离, 并对称赋值
                #sim_d = 1 - get_distance(u1,u2,users_num,users_cat)
                sim_r = get_user_sim(u1,u2,user_movie_rating)
                #print(sim_d,':', sim_r)
                user_distance[u1,u2] = sim_r
                user_distance[u2,u1] = sim_r
                
        dump(user_distance, self.output_dir+'{0}.joblib.gz'.format(self.k), compress=('gzip',3))

    def get_u_d(self,):
        users = load(self.output_dir + 'users.joblib.gz')
        users.index = users.pop('UserID')
        print('getting the distance')
        num_users = users.index.max()+1
        # 生成数值型和类别型数据
        users_num = users.loc[:,'Gender_0':]
        users_cat = users.loc[:,'Gender_0':]
        # 生成空的distance, 默认为1
        user_distance = np.ones((num_users, num_users), dtype=np.float64)
        # 生成user_index
        u_index = users.index
        for i,u1 in enumerate(u_index):
            # 显示进度
            if i%100 == 0: print(i)
            # 对角线距离为0
            user_distance[u1,u1] = 0
            for u2 in u_index[i+1:]:
                # 计算距离, 并对称赋值
                sim_d = 1 - get_distance(u1,u2,users_num,users_cat)
                #sim_r = get_user_sim(u1,u2,user_movie_rating)
                #print(sim_d,':', sim_r)
                user_distance[u1,u2] = sim_d
                user_distance[u2,u1] = sim_d
                
        dump(user_distance, self.output_dir+'{0}.joblib.gz'.format(self.k), compress=('gzip',3))
    
    def get_m_s(self,):
        movies = load(output_dir + 'movies.joblib.gz')
        movies.index = movies.pop('MovieID')
        print('getting the distance')
        num_movies = movies.index.max()+1
        # 生成数值型和类别型数据
        movies_num = movies.loc[:,'Genres_Animation':'Genres_Western']
        movies_cat = movies.loc[:,'Title_yeay':]
        # 生成空的distance, 默认为1
        movie_distance = np.ones((num_movies, num_movies), dtype=np.float64)
        # 生成movie_index
        m_index = movies.index
        for i,m1 in enumerate(m_index):
            # 显示进度
            if i%100 == 0: print(i)
            # 对角线距离为0
            movie_distance[m1,m1] = 0
            for m2 in m_index[i+1:]:
                # 计算距离, 并对称赋值
                #sim_d = 1 - get_distance(m1,m2,movies_num,movies_cat)
                sim_r = get_movie_sim(m1,m2,user_movie_rating)
                #print(sim_d,':',sim_r)
                movie_distance[m1,m2] = sim_r
                movie_distance[m2,m1] = sim_r
        
        dump(movie_distance, self.output_dir+'{0}.joblib.gz'.format(self.k), compress=('gzip',3))
    
    def get_m_d(self,):
        movies = load(output_dir + 'movies.joblib.gz')
        movies.index = movies.pop('MovieID')
        print('getting the distance')
        num_movies = movies.index.max()+1
        # 生成数值型和类别型数据
        movies_num = movies.loc[:,'Genres_Animation':'Genres_Western']
        movies_cat = movies.loc[:,'Title_yeay':]
        # 生成空的distance, 默认为1
        movie_distance = np.ones((num_movies, num_movies), dtype=np.float64)
        # 生成movie_index
        m_index = movies.index
        for i,m1 in enumerate(m_index):
            # 显示进度
            if i%100 == 0: print(i)
            # 对角线距离为0
            movie_distance[m1,m1] = 0
            for m2 in m_index[i+1:]:
                # 计算距离, 并对称赋值
                sim_d = 1 - get_distance(m1,m2,movies_num,movies_cat)
                #sim_r = get_movie_sim(m1,m2,user_movie_rating)
                #print(sim_d,':',sim_r)
                movie_distance[m1,m2] = sim_d
                movie_distance[m2,m1] = sim_d
        
        dump(movie_distance, self.output_dir+'{0}.joblib.gz'.format(self.k), compress=('gzip',3))


### 推荐同类型的电影
思路是计算当前看的电影特征向量与整个电影特征矩阵的相似度，取相似度最大的top_k个，这里加了些随机选择在里面，保证每次的推荐稍稍有些不同。

In [59]:
def recommend_same_type_movie(movie_id, top_k = 20):
    
    movies_sim = movie_distance
    movie_sim = pd.Series(movies_sim[movie_id], index=np.arange(movies_sim.shap[0]))
    movie_sim_sorted = movie_sim.sort_value()
    top_20 = movie_sim_sorted[:top_k].index

    movies_orig = load
    print("您看的电影是：{}".format(movies_orig[movie_id_val]))
    print("以下是给您的推荐：")
    results = top_20[np.random.randint(0,20,(5))]
    for m in results:
        print(m, movies[m])
    
    return results

In [60]:
recommend_same_type_movie(1401, 20)

您看的电影是：[1401 'Ghosts of Mississippi (1996)' 'Drama']
以下是给您的推荐：
3385
[3454 'Whatever It Takes (2000)' 'Comedy|Romance']
707
[716 'Switchblade Sisters (1975)' 'Crime']
2351
[2420 'Karate Kid, The (1984)' 'Drama']
2189
[2258 'Master Ninja I (1984)' 'Action']
2191
[2260 'Wisdom (1986)' 'Action|Crime']


{707, 2189, 2191, 2351, 3385}

### 推荐您喜欢的电影
思路是使用用户特征向量与电影特征矩阵计算所有电影的评分，取评分最高的top_k个，同样加了些随机选择部分。

In [61]:
def recommend_your_favorite_movie(user_id_val, top_k = 10):

    loaded_graph = tf.Graph()  #
    with tf.Session(graph=loaded_graph) as sess:  #
        # Load saved model
        loader = tf.train.import_meta_graph(load_dir + '.meta')
        loader.restore(sess, load_dir)

        #推荐您喜欢的电影
        probs_embeddings = (users_matrics[user_id_val-1]).reshape([1, 200])

        probs_similarity = tf.matmul(probs_embeddings, tf.transpose(movie_matrics))
        sim = (probs_similarity.eval())
    #     print(sim.shape)
    #     results = (-sim[0]).argsort()[0:top_k]
    #     print(results)
        
    #     sim_norm = probs_norm_similarity.eval()
    #     print((-sim_norm[0]).argsort()[0:top_k])
    
        print("以下是给您的推荐：")
        p = np.squeeze(sim)
        p[np.argsort(p)[:-top_k]] = 0
        p = p / np.sum(p)
        results = set()
        while len(results) != 5:
            c = np.random.choice(3883, 1, p=p)[0]
            results.add(c)
        for val in (results):
            print(val)
            print(movies_orig[val])

        return results


In [62]:
recommend_your_favorite_movie(234, 10)

以下是给您的推荐：
1642
[1688 'Anastasia (1997)' "Animation|Children's|Musical"]
994
[1007 'Apple Dumpling Gang, The (1975)' "Children's|Comedy|Western"]
667
[673 'Space Jam (1996)' "Adventure|Animation|Children's|Comedy|Fantasy"]
1812
[1881 'Quest for Camelot (1998)' "Adventure|Animation|Children's|Fantasy"]
1898
[1967 'Labyrinth (1986)' "Adventure|Children's|Fantasy"]


{667, 994, 1642, 1812, 1898}

### 看过这个电影的人还看了（喜欢）哪些电影
- 首先选出喜欢某个电影的top_k个人，得到这几个人的用户特征向量。
- 然后计算这几个人对所有电影的评分
- 选择每个人评分最高的电影作为推荐
- 同样加入了随机选择

In [63]:
import random

def recommend_other_favorite_movie(movie_id_val, top_k = 20):
    loaded_graph = tf.Graph()  #
    with tf.Session(graph=loaded_graph) as sess:  #
        # Load saved model
        loader = tf.train.import_meta_graph(load_dir + '.meta')
        loader.restore(sess, load_dir)

        probs_movie_embeddings = (movie_matrics[movieid2idx[movie_id_val]]).reshape([1, 200])
        probs_user_favorite_similarity = tf.matmul(probs_movie_embeddings, tf.transpose(users_matrics))
        favorite_user_id = np.argsort(probs_user_favorite_similarity.eval())[0][-top_k:]
    #     print(normalized_users_matrics.eval().shape)
    #     print(probs_user_favorite_similarity.eval()[0][favorite_user_id])
    #     print(favorite_user_id.shape)
    
        print("您看的电影是：{}".format(movies_orig[movieid2idx[movie_id_val]]))
        
        print("喜欢看这个电影的人是：{}".format(users_orig[favorite_user_id-1]))
        probs_users_embeddings = (users_matrics[favorite_user_id-1]).reshape([-1, 200])
        probs_similarity = tf.matmul(probs_users_embeddings, tf.transpose(movie_matrics))
        sim = (probs_similarity.eval())
    #     results = (-sim[0]).argsort()[0:top_k]
    #     print(results)
    
    #     print(sim.shape)
    #     print(np.argmax(sim, 1))
        p = np.argmax(sim, 1)
        print("喜欢看这个电影的人还喜欢看：")

        results = set()
        while len(results) != 5:
            c = p[random.randrange(top_k)]
            results.add(c)
        for val in (results):
            print(val)
            print(movies_orig[val])
        
        return results

In [64]:
recommend_other_favorite_movie(1401, 20)

您看的电影是：[1401 'Ghosts of Mississippi (1996)' 'Drama']
喜欢看这个电影的人是：[[5782 'F' 35 0]
 [5767 'M' 25 2]
 [3936 'F' 35 12]
 [3595 'M' 25 0]
 [1696 'M' 35 7]
 [2728 'M' 35 12]
 [763 'M' 18 10]
 [4404 'M' 25 1]
 [3901 'M' 18 14]
 [371 'M' 18 4]
 [1855 'M' 18 4]
 [2338 'M' 45 17]
 [450 'M' 45 1]
 [1130 'M' 18 7]
 [3035 'F' 25 7]
 [100 'M' 35 17]
 [567 'M' 35 20]
 [5861 'F' 50 1]
 [4800 'M' 18 4]
 [3281 'M' 25 17]]
喜欢看这个电影的人还喜欢看：
1779
[1848 'Borrowers, The (1997)' "Adventure|Children's|Comedy|Fantasy"]
1244
[1264 'Diva (1981)' 'Action|Drama|Mystery|Romance|Thriller']
1812
[1881 'Quest for Camelot (1998)' "Adventure|Animation|Children's|Fantasy"]
1742
[1805 'Wild Things (1998)' 'Crime|Drama|Mystery|Thriller']
2535
[2604 'Let it Come Down: The Life of Paul Bowles (1998)' 'Documentary']


{1244, 1742, 1779, 1812, 2535}

# 结论

以上就是实现的常用的推荐功能，将网络模型作为回归问题进行训练，得到训练好的用户特征矩阵和电影特征矩阵进行推荐。